Prepocessing

In [1]:
from google.colab import files
uploaded = files.upload()

Saving charity_data.csv to charity_data.csv


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import io

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv(io.BytesIO(uploaded['charity_data.csv']))
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN"])

In [4]:
application_df = application_df.drop(columns=["NAME"])

In [5]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# CLASSIFICATION counts > 1
classification_bins = application_df["CLASSIFICATION"].value_counts()
class_data = classification_bins[classification_bins>1]
class_data

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace'
classifications_to_replace = list(classification_bins[classification_bins <1800].index)

# Replace in dataframe
for c in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(c,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummy = pd.get_dummies(application_df)
application_df_dummy.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [13]:
# Split our preprocessed data into our features and target arrays
X = application_df_dummy.drop('IS_SUCCESSFUL', axis=1)
y = application_df_dummy['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
pd.DataFrame(X_train_scaled)

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0.014433,-0.031834,-0.090256,-0.124074,-0.179818,0.520115,-0.220998,-0.187214,-0.192047,-0.14732,...,-0.147467,-0.127491,-0.326902,-0.084446,-0.169952,-0.351413,-0.064996,-0.071466,0.029588,-0.029588
1,0.014433,-0.031834,-0.090256,-0.124074,-0.179818,0.520115,-0.220998,-0.187214,-0.192047,-0.14732,...,-0.147467,-0.127491,-0.326902,-0.084446,-0.169952,-0.351413,-0.064996,-0.071466,0.029588,-0.029588
2,0.014433,-0.030578,-0.090256,-0.124074,-0.179818,0.520115,-0.220998,-0.187214,-0.192047,-0.14732,...,-0.147467,-0.127491,-0.326902,-0.084446,-0.169952,2.845657,-0.064996,-0.071466,0.029588,-0.029588
3,0.014433,-0.030712,-0.090256,-0.124074,-0.179818,0.520115,-0.220998,-0.187214,-0.192047,-0.14732,...,-0.147467,-0.127491,3.059025,-0.084446,-0.169952,-0.351413,-0.064996,-0.071466,0.029588,-0.029588
4,0.014433,-0.031834,-0.090256,-0.124074,-0.179818,0.520115,-0.220998,-0.187214,-0.192047,-0.14732,...,-0.147467,-0.127491,-0.326902,-0.084446,-0.169952,-0.351413,-0.064996,-0.071466,0.029588,-0.029588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24004,0.014433,-0.028562,-0.090256,-0.124074,-0.179818,0.520115,-0.220998,-0.187214,-0.192047,-0.14732,...,-0.147467,-0.127491,-0.326902,-0.084446,-0.169952,2.845657,-0.064996,-0.071466,0.029588,-0.029588
24005,0.014433,-0.031834,-0.090256,-0.124074,-0.179818,0.520115,-0.220998,-0.187214,-0.192047,-0.14732,...,-0.147467,-0.127491,-0.326902,-0.084446,-0.169952,-0.351413,-0.064996,-0.071466,0.029588,-0.029588
24006,0.014433,-0.031834,-0.090256,-0.124074,-0.179818,0.520115,-0.220998,-0.187214,-0.192047,-0.14732,...,-0.147467,-0.127491,-0.326902,-0.084446,-0.169952,-0.351413,-0.064996,-0.071466,0.029588,-0.029588
24007,0.014433,-0.031834,-0.090256,-0.124074,-0.179818,0.520115,-0.220998,-0.187214,-0.192047,-0.14732,...,-0.147467,-0.127491,-0.326902,-0.084446,-0.169952,-0.351413,-0.064996,-0.071466,0.029588,-0.029588


In [16]:
X_train_scaled[0]

array([ 0.01443255, -0.03183394, -0.09025589, -0.12407407, -0.17981755,
        0.52011454, -0.22099809, -0.18721417, -0.19204656, -0.14731955,
       -0.14687625,  1.08758209, -0.04333378, -1.08067365, -0.0295878 ,
       -0.01117894, -0.01707754, -1.01168895, -0.39883726,  2.14729839,
       -0.24399224, -0.24609325, -0.26460377, -0.10704627, -0.0653187 ,
       -0.00912738,  0.4719144 , -0.44691181, -0.6550627 , -0.11841472,
       -0.03477558,  0.67859435,  0.63628346, -0.14746705, -0.12749105,
       -0.32690151, -0.08444626, -0.16995191, -0.35141268, -0.06499636,
       -0.07146594,  0.0295878 , -0.0295878 ])

## Compile, Train, and Evaluate Model

Input layer = equal to the number of features (columns) in your data = 43

Output layer = If the NN is a classifier, then it also has a single node unless softmax is used in which case the output layer has one node per class label in your model.

Neurons in the first hidden layer = mean of the neurons in the input and output layers // halved features

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
def create_model(hp):
  nn = tf.keras.models.Sequential()
  activation = hp.Choice('activation', ['relu', 'tanh'])
  nn.add(tf.keras.layers.Dense(units=hp.Int('first_units', 
                                           min_value = 10,
                                           max_value = 60,
                                           step=5), activation=activation, input_dim=43))
  for i in range(hp.Int('num_layers', 1, 5)):
    nn.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                              min_value=10,
                                              max_value=60, 
                                              step=5), activation=activation))
  nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))
  # Compile the model
  nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return nn


In [18]:
pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 4.8 MB/s 


In [19]:
pip install autokeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 162 kB 5.1 MB/s 


In [20]:
# Import the kerastuner library
import keras_tuner as kt
from tensorflow import keras

#tuner = kt.Hyperband(
    #create_model,
    #="val_accuracy",
   # max_trials=100,
   # hyperband_iterations=2)
tuner = kt.RandomSearch(
    hypermodel=create_model,
    objective="val_accuracy",
    max_trials=30)

In [21]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 30 Complete [00h 00m 43s]
val_accuracy: 0.7322643399238586

Best val_accuracy So Far: 0.7340136170387268
Total elapsed time: 00h 31m 49s


In [22]:
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 40, 'num_layers': 2, 'units_0': 30, 'units_1': 35, 'units_2': 30, 'units_3': 35, 'units_4': 50}
{'activation': 'relu', 'first_units': 20, 'num_layers': 3, 'units_0': 55, 'units_1': 25, 'units_2': 20, 'units_3': 30, 'units_4': 40}
{'activation': 'tanh', 'first_units': 15, 'num_layers': 5, 'units_0': 50, 'units_1': 60, 'units_2': 35, 'units_3': 25, 'units_4': 55}
